# **Study on the Minimum Gap between Highway Cars for the Ego to Merge into the Highway**

##### This project aims to determine the minimum gap that two highway vehicles should maintain to allow a merging car (ego vehicle) to safely merge between them. The process begins by evaluating a range of gap intervals to assess which performs best in terms of safety and traffic flow. This is done by adjusting the 'minimum_gap_range' in the environment's configuration. Subsequently, the range of this gap interval will be gradually narrowed down to identify the exact gap(s) that offer the safest and most efficient merging conditions, with the goal of pinpointing a single optimal gap or multiple gaps if no clear standout emerges.

### **Imports**

In [1]:
import gymnasium as gym
from matplotlib import pyplot as plt
import pprint
from IPython.display import Video
import cv2
import imageio
import highway_env
import pandas as pd
import time
import numpy as np
from stable_baselines3 import PPO
from highway_env import utils
from highway_env.envs import MergeEnv
from highway_env.vehicle.controller import ControlledVehicle
%matplotlib inline

### **Creation of the environment**

##### With the ego-vehicle on the merging lane and two vehicles on the highway, on the right most lane and with a certain distance between them

In [2]:
class RightLaneVehicle(ControlledVehicle):
    """
    Um veículo que é restrito a ficar na lane da direita e nunca muda de lane.
    """
    def act(self, action: int = None) -> None:
        # Assegura que o veículo não mude de lane (desautoriza ações 0 e 2 para mudança de lane)
        if action in [0, 2]:  # Ações para mudar para a esquerda ou direita
            action = 1  # Forçar a manter a lane (ação 1)
        super().act(action)


class CustomMergeEnv(MergeEnv):
    def __init__(self, distance, *args, **kwargs):
        # Set the distance attribute first
        self.distance = distance  # Set the distance before calling the base class
        # Initialize the base class (MergeEnv) after setting the distance
        super().__init__(*args, **kwargs)  # This ensures the parent class is properly initialized

    def _make_vehicles(self) -> None:
        # Ensure that distance is properly initialized before use
        road = self.road

        # Ponto de mesclagem (merge) na lane 0
        merge_position = road.network.get_lane(("b", "c", 0)).position(0, 0)  # Ponto de mesclagem na autoestrada
        
        # Posição inicial do veículo ego na lane de mesclagem
        ego_initial_position = road.network.get_lane(("j", "k", 0)).position(30, 0)  # Ego vehicle na lane de mesclagem

        # Ajustar a posição inicial do veículo da autoestrada com base na distância fornecida
        highway_vehicle_initial_position = road.network.get_lane(("a", "b", 1)).position(100 - self.distance, 0)  # Ajuste na posição de acordo com a distância
        highway_vehicle_initial_position_1 = road.network.get_lane(("a", "b", 1)).position(100 - self.distance, 0)  # Outro veículo ajustado pela mesma distância

        # Definir velocidades iniciais
        ego_speed = 20  # Velocidade inicial do ego

        # Calcular o tempo para ambos os veículos chegarem ao ponto de mesclagem
        time_to_merge = (merge_position[0] - ego_initial_position[0]) / ego_speed

        # Ajustar a velocidade do veículo da autoestrada para garantir que ambos cheguem ao mesmo tempo
        highway_vehicle_speed = (merge_position[0] - highway_vehicle_initial_position[0]) / time_to_merge

        # Criar o veículo ego na lane de mesclagem
        ego_vehicle = self.action_type.vehicle_class(
            road, ego_initial_position, speed=ego_speed
        )
        road.vehicles.append(ego_vehicle)

        # Criar o veículo na lane da direita da autoestrada (lane 1)
        highway_vehicle = RightLaneVehicle(
            road, highway_vehicle_initial_position, speed=highway_vehicle_speed
        )
        road.vehicles.append(highway_vehicle)

        # Criar o segundo veículo na mesma lane
        highway_vehicle_1 = RightLaneVehicle(
            road, highway_vehicle_initial_position_1, speed=highway_vehicle_speed
        )
        road.vehicles.append(highway_vehicle_1)

        # Definir o veículo ego como o veículo principal
        self.vehicle = ego_vehicle

    def set_distance(self, distance: int) -> None:
        """
        Método para alterar a distância entre os veículos no ambiente.
        """
        self.distance = distance  # Atualiza a distância
        self._make_vehicles()  # Recria os veículos com a nova distância

In [3]:
# Registering the custom environment
gym.envs.registration.register(
    id='CustomMerge-v0',
    entry_point='__main__:CustomMergeEnv', 
)

#### **With distance 20**

In [4]:
env_20 = gym.make("CustomMerge-v0", render_mode='rgb_array', distance=20) 

#### **With distance 40**

In [5]:
env_40 = gym.make("CustomMerge-v0", render_mode='rgb_array', distance=40) 

#### **With distance 60**

In [6]:
env_60 = gym.make("CustomMerge-v0", render_mode='rgb_array', distance=60) 

#### **With distance 80**

In [7]:
env_80 = gym.make("CustomMerge-v0", render_mode='rgb_array', distance=80) 

#### **With distance 100**

In [8]:
env_100 = gym.make("CustomMerge-v0", render_mode='rgb_array', distance=100) 

### **Training the models**

In [ ]:
model = PPO('MlpPolicy', env_20,
            policy_kwargs=dict(net_arch=[256, 256]),
            learning_rate=5e-4,
            n_steps=2048, 
            batch_size=64, 
            n_epochs=10,  
            gamma=0.8,
            gae_lambda=0.95, 
            clip_range=0.2, 
            verbose=1,
            tensorboard_log="env_minimum_gap_20/")
timesteps = 1000000
model.learn(total_timesteps=timesteps)
model.save("env_minimum_gap_20/model")

In [ ]:
model = PPO('MlpPolicy', env_40,
            policy_kwargs=dict(net_arch=[256, 256]),
            learning_rate=5e-4,
            n_steps=2048, 
            batch_size=64, 
            n_epochs=10,  
            gamma=0.8,
            gae_lambda=0.95, 
            clip_range=0.2, 
            verbose=1,
            tensorboard_log="env_minimum_gap_40/")
timesteps = 1000000
model.learn(total_timesteps=timesteps)
model.save("env_minimum_gap_40/model")

In [ ]:
model = PPO('MlpPolicy', env_60,
            policy_kwargs=dict(net_arch=[256, 256]),
            learning_rate=5e-4,
            n_steps=2048, 
            batch_size=64, 
            n_epochs=10,  
            gamma=0.8,
            gae_lambda=0.95, 
            clip_range=0.2, 
            verbose=1,
            tensorboard_log="env_minimum_gap_60/")
timesteps = 1000000
model.learn(total_timesteps=timesteps)
model.save("env_minimum_gap_60/model")

In [ ]:
model = PPO('MlpPolicy', env_80,
            policy_kwargs=dict(net_arch=[256, 256]),
            learning_rate=5e-4,
            n_steps=2048, 
            batch_size=64, 
            n_epochs=10,  
            gamma=0.8,
            gae_lambda=0.95, 
            clip_range=0.2, 
            verbose=1,
            tensorboard_log="env_minimum_gap_80/")
timesteps = 1000000
model.learn(total_timesteps=timesteps)
model.save("env_minimum_gap_80/model")

In [ ]:
model = PPO('MlpPolicy', env_100,
            policy_kwargs=dict(net_arch=[256, 256]),
            learning_rate=5e-4,
            n_steps=2048, 
            batch_size=64, 
            n_epochs=10,  
            gamma=0.8,
            gae_lambda=0.95, 
            clip_range=0.2, 
            verbose=1,
            tensorboard_log="env_minimum_gap_100/")
timesteps = 1000000
model.learn(total_timesteps=timesteps)
model.save("env_minimum_gap_100/model")

### **Evaluate and compare the models**

**For the minimum gap 20**
- Average Reward:
- Average Steps to Merge: 
- Average Episode Time: 
- Number of Collisions: 
- Successful Merges: 
- Number of Dangerous Driving Episodes (sudden speed changes): 

**For the minimum gap 40**
- Average Reward: 
- Average Steps to Merge: 
- Average Episode Time: 
- Number of Collisions: 
- Successful Merges: 
- Number of Dangerous Driving Episodes (sudden speed changes): 

**For the minimum gap 60**
- Average Reward: 
- Average Steps to Merge: 
- Average Episode Time: 
- Number of Collisions:
- Successful Merges: 
- Number of Dangerous Driving Episodes (sudden speed changes): 

**For the minimum gap 80**
- Average Reward: 
- Average Steps to Merge: 
- Average Episode Time: 
- Number of Collisions: 
- Successful Merges: 
- Number of Dangerous Driving Episodes (sudden speed changes): 

**For the minimum gap 100**
- Average Reward: 
- Average Steps to Merge: 
- Average Episode Time: 
- Number of Collisions: 
- Successful Merges: 
- Number of Dangerous Driving Episodes (sudden speed changes): 

In [ ]:
def evaluate_agent(model, env, num_episodes, speed_threshold_ratio=0.3):
    """
    Função para avaliar um modelo em um ambiente específico.

    Parâmetros:
    - model: o modelo de aprendizado a ser avaliado.
    - env: o ambiente no qual o modelo será avaliado.
    - num_episodes: número de episódios para a avaliação.
    - speed_threshold_ratio: fator que determina o threshold de velocidade (default 0.3).

    Retorno:
    - Um dicionário com as métricas de avaliação: recompensa média, número de colisões, número de fusões bem-sucedidas, etc.
    """
    total_rewards = []  # Lista para armazenar as recompensas totais por episódio
    total_collisions = 0  # Contador para colisões
    successful_merges = 0  # Contador para fusões bem-sucedidas
    dangerous_driving_episodes = 0  # Contador para episódios com direção perigosa
    total_steps_to_merge = []  # Lista para armazenar os passos até a fusão
    total_episode_times = []  # Lista para armazenar o tempo de cada episódio

    # Cálculo do threshold de velocidade
    reward_speed_range = env.config["reward_speed_range"]
    speed_threshold = (reward_speed_range[1] - reward_speed_range[0]) * speed_threshold_ratio  # Limite para mudanças repentinas de velocidade

    for episode in range(num_episodes):
        start_time = time.time()  # Registrar o tempo de início do episódio
        obs, info = env.reset()  # Resetar o ambiente e pegar a observação inicial
        done = False  # Variável para verificar se o episódio terminou
        episode_reward = 0  # Variável para acumular a recompensa do episódio
        collisions = 0  # Contador de colisões neste episódio
        dangerous_driving = False  # Flag para direção perigosa
        steps_to_merge = 0  # Contador de passos até a fusão
        last_speed = None  # Inicializar a velocidade anterior como None

        # Armazenar as posições dos veículos na rodovia para verificar a fusão
        highway_vehicle_positions = []

        while not done:  # Loop até o episódio terminar
            # O agente escolhe uma ação
            action, _states = model.predict(obs, deterministic=True)
            # Executar a ação no ambiente
            obs, reward, terminated, truncated, info = env.step(action)

            # Armazenar as posições dos veículos na rodovia
            if 'highway_vehicles' in info:
                highway_vehicle_positions = [vehicle.position[0] for vehicle in info['highway_vehicles']]

            dangerous_driving = False  # Resetar flag de direção perigosa
            episode_reward += reward  # Acumular recompensa do episódio
            steps_to_merge += 1  # Incrementar os passos até a fusão

            # Verificar a velocidade atual e arredondar para 2 casas decimais
            current_speed = round(info.get('speed', 0), 2)

            # Verificar mudanças repentinas de velocidade
            if last_speed is not None and abs(current_speed - last_speed) > speed_threshold:
                dangerous_driving = True  # Marcar como direção perigosa se a mudança de velocidade for acima do threshold

            last_speed = current_speed  # Atualizar a velocidade anterior para a próxima iteração

            # Verificar colisões
            if 'crashed' in info and info['crashed']:
                collisions += 1  # Incrementar o contador de colisões

            # Verificar se o episódio terminou (terminado ou truncado)
            done = terminated or truncated

        # Considerar uma fusão bem-sucedida se o veículo ego estiver entre os dois veículos da rodovia
        if highway_vehicle_positions:
            ego_vehicle_position = info['ego_vehicle_position'][0]  # Coordenada x do veículo ego
            highway_vehicle_position_1 = highway_vehicle_positions[0]
            highway_vehicle_position_2 = highway_vehicle_positions[1]

            # Verificar se o veículo ego está entre os dois veículos da rodovia
            if (highway_vehicle_position_1 < ego_vehicle_position < highway_vehicle_position_2) or \
               (highway_vehicle_position_2 < ego_vehicle_position < highway_vehicle_position_1):
                successful_merges += 1  # Incrementar fusões bem-sucedidas se o veículo ego estiver entre os veículos da rodovia

        # Logar as métricas do episódio
        total_rewards.append(episode_reward)  # Adicionar recompensa do episódio à lista
        total_collisions += collisions  # Atualizar o total de colisões
        total_steps_to_merge.append(steps_to_merge)  # Adicionar passos para fusão

        if dangerous_driving:
            dangerous_driving_episodes += 1  # Incrementar contagem de episódios com direção perigosa

        # Calcular o tempo do episódio
        episode_time = time.time() - start_time  # Calcular o tempo total do episódio
        total_episode_times.append(episode_time)  # Adicionar o tempo do episódio à lista

    # Calcular métricas finais
    avg_reward = np.mean(total_rewards)  # Recompensa média
    avg_steps_to_merge = np.mean(total_steps_to_merge)  # Passos médios até a fusão
    avg_episode_time = np.mean(total_episode_times)  # Tempo médio de episódio

    # Exibir resultados
    print(f"Average Reward: {avg_reward}")  # Exibir recompensa média
    print(f"Average Steps to Merge: {avg_steps_to_merge}")  # Exibir passos médios até a fusão
    print(f"Average Episode Time: {avg_episode_time:.2f} seconds")  # Exibir tempo médio de episódio
    print(f"Number of Collisions: {total_collisions}")  # Exibir número de colisões
    print(f"Successful Merges: {successful_merges}")  # Exibir número de fusões bem-sucedidas
    print(f"Number of Dangerous Driving Episodes (sudden speed changes): {dangerous_driving_episodes}")  # Exibir episódios com direção perigosa

    return {
        "avg_reward": avg_reward,  # Recompensa média
        "avg_steps_to_merge": avg_steps_to_merge,  # Passos médios até a fusão
        "avg_episode_time": avg_episode_time,  # Tempo médio de episódio
        "number_collisions": total_collisions,  # Número total de colisões
        "successful_merges": successful_merges,  # Número de fusões bem-sucedidas
        "number_dangerous_episodes": dangerous_driving_episodes  # Número de episódios com direção perigosa
    }

In [ ]:
# Load the trained model
model = PPO.load("env_minimum_gap_20/model")  

# Evaluate the model
results = evaluate_agent(model, env_20, 200) 

In [ ]:
# Load the trained model
model = PPO.load("env_minimum_gap_40/model")  

# Evaluate the model
results = evaluate_agent(model, env_40, 200) 

In [ ]:
# Load the trained model
model = PPO.load("env_minimum_gap_60/model")  

# Evaluate the model
results = evaluate_agent(model, env_60, 200) 

In [ ]:
# Load the trained model
model = PPO.load("env_minimum_gap_80/model")  

# Evaluate the model
results = evaluate_agent(model, env_80, 200) 

In [ ]:
# Load the trained model
model = PPO.load("env_minimum_gap_100/model")  

# Evaluate the model
results = evaluate_agent(model, env_100, 200) 

Textinho a analisar os resultados

### **Narrowing the XXXXXXXX distance interval**

MUDAR ESTE TEXTO PARA FALAR ACERCA DO INTERVALO ÓTIMO E DIZER QUE É DE 2 EM 2

##### In order to identify the optimal speed range within the broader interval of (10, 20), a process of gradual refinement was employed. By breaking this interval into smaller subintervals of 0.5, such as (10, 10.5), (10.5, 11), and so on, each subinterval is evaluated independently using the same metrics as before. The goal of this narrowing process is to identify which specific speed range yields the highest rewards, minimizes collisions, and reduces dangerous driving episodes. By successively refining these subintervals and analyzing the results, we can pinpoint the exact optimal speed range where the agent performs most efficiently and safely. This step-by-step method ensures that performance is maximized within the interval of interest.

In [ ]:
def evaluate_distance_intervals(model, env, base_interval, step_size, num_episodes):
    """
    Função para avaliar o agente em diferentes intervalos de distância entre os veículos.
    
    model: modelo do agente (PPO, etc.)
    env: o ambiente onde a simulação ocorre
    base_interval: intervalo inicial de distância (ex. 40)
    step_size: decremento para cada subintervalo de distância (ex. 2 metros)
    num_episodes: número de episódios de simulação para cada subintervalo
    
    Returns: um dicionário com os resultados para cada subintervalo.
    """
    # Criação dos subintervalos de distância (decrementando de 'step_size' em 'step_size')
    subintervals = list(range(base_interval, 0, -step_size))  # Lista de distâncias, de base_interval até 0 ou o mínimo desejado
    
    results = {}  # Dicionário para armazenar os resultados

    for distance in subintervals:
        print(f"Evaluando para a distância de {distance} metros")  # Imprimir a distância que está sendo avaliada
        
        # Ajustar o ambiente com a nova distância
        env = gym.make("CustomMerge-v0", render_mode='rgb_array', distance=distance)  # Criar o ambiente com a distância ajustada
        
        result = evaluate_agent(model, env, num_episodes)  # Avaliar o agente para essa distância
        results[distance] = result  # Armazenar o resultado para essa distância
        
        print("\n")  # Espaçamento entre os resultados

    return results  # Retornar os resultados para todos os intervalos de distância

In [ ]:
# Parâmetros para o intervalo base e tamanho do subintervalo
base_interval = 40  # Distância inicial de 40 metros   MUDAR AQUI
step_size = 2  # O tamanho de cada subintervalo (reduzir a distância em 2 metros)
num_episodes = 200  # Número de episódios para avaliar

# Carregar o modelo
model = PPO.load("modelo_40", custom_objects={"observation_space": env_40.observation_space, "action_space": env_40.action_space}) MUDAR AQUI

# Avaliar o modelo nos subintervalos de distância
results = evaluate_distance_intervals(model, env_40, base_interval, step_size, num_episodes)  # Chama a função para avaliar os modelos MUDAR AQUI

In [ ]:
# Print the results of the evaluations
print(f"Results: {results}")

In [ ]:
# Create a list to hold the data
data = [] 

# Iterate through the results and flatten the structure
for speed_range, metrics in results.items():  # Loop through each speed range and its corresponding metrics
    row = {'Speed Range': f"{speed_range[0]} - {speed_range[1]}"}  # Create a new dictionary for the current row with the speed range
    row.update(metrics)  # Add metrics to the row dictionary
    data.append(row)  # Append the row to the data list

# Create a DataFrame from the list of dictionaries
results_df = pd.DataFrame(data)  # Convert the list of dictionaries into a Pandas DataFrame

results_df  # Display the DataFrame

TEXTINHO A ANALISAR OS RESULTADOS

MUDAR O CÓDIGO PARA GERAR VIDEO DO MODELO MELHOR

In [ ]:
# Load the trained model
model = PPO.load("vel_study_10_20/model")

# Initialize the environment and variables for recording
frames = []
obs, info = env_10_20.reset()
done = False
step_count = 0
max_steps = 1000

# Resize frame to be divisible by 16 (macro block size for video codecs)
def resize_frame_to_macro_block_size(frame, block_size=16):
    h, w, _ = frame.shape
    new_w = (w // block_size) * block_size
    new_h = (h // block_size) * block_size
    return cv2.resize(frame, (new_w, new_h))

# Run the agent in the environment
while step_count < max_steps and not done:
    action, _ = model.predict(obs)
    obs, reward, done, truncated, info = env_10_20.step(action)
    frame = env_10_20.render()

    # Resize the frame to avoid the macro_block_size warning
    resized_frame = resize_frame_to_macro_block_size(frame)
    frames.append(resized_frame)
    
    step_count += 1

# Close the environment
env_10_20.close()

# Save the frames as a video
video_filename = "velocity_study.mp4"
imageio.mimsave(video_filename, frames, fps=30)
print(f"Video saved as {video_filename}")

In [ ]:
# Display the video
video_filename = "velocity_study.mp4"
Video(video_filename, embed=True)